# Housing Regression Maching Learning Project

### Imports (Data and Packages)

In [19]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from data_preprocessing_final import cleaning, dummify, scale_data, ord_encoding, initiate_data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
%matplotlib inline 

In [20]:
housing = pd.read_csv('./data/Ames_Housing_Price_Data.csv', index_col=0)
housing = housing[housing['SaleCondition'] == 'Normal']

In [6]:
train_data_linear, train_target_linear, test_data_linear, test_target_linear, train_data_tree, train_target_tree, test_data_tree, test_target_tree = initiate_data(housing)

### Lasso Regression

In [25]:
## Working on Lasso Regression
from sklearn.linear_model import Lasso
lasso = Lasso()

### Grid Search

In [26]:
alphaRange = np.linspace(0,0.002,5)
params = {'alpha': alphaRange, 'max_iter' : [10000]}

In [27]:
gs = GridSearchCV(lasso, params, scoring = 'r2')
gs.fit(train_data_linear, train_target_linear)

C:\Users\jackc\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 184241756562.63745, tolerance: 776124972.1245021
  model = cd_fast.enet_coordinate_descent(
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator

GridSearchCV(estimator=Lasso(),
             param_grid={'alpha': array([0.    , 0.0005, 0.001 , 0.0015, 0.002 ]),
                         'max_iter': [10000]},
             scoring='r2')

In [28]:
gs.best_params_

{'alpha': 0.0, 'max_iter': 10000}

In [29]:
gs.score(train_data_linear, train_target_linear)

0.9501590349539748

In [30]:
gs.score(test_data_linear, test_target_linear)


0.9316675274727789